# Coroutines
- Syntactically like generators - a function with a `yield` keyword in its body.
- In a coroutine, however, the yield keyword appears in the right side of an expression, e.g. `datum = yield` and it may/may not produce a value.
- Yields `None` if there's no expression after the `yield` keyword.
- Coroutine may receive data from it's caller using the `.send(data)` instead of `next(..)`

In [6]:
"""
Define a coroutine as a generator function: with yield in it's body
"""
def simple_coroutine():
    print('-> Coroutine started')
    x = yield
    print('-> Coroutine received', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x1108b3888>

In [4]:
"""
First call is `next` since the generator hasn't started and so it's not waiting
in a yield so we can't send it any data initially.
"""
next(my_coro)

-> Coroutine started


In [5]:
"""
This call makes the `yield` in the generator evaluate to `42`.
The generator resumes and runs until the next yield or until termination.
"""
my_coro.send(42)

-> Coroutine received 42


StopIteration: 

In [7]:
"""A coroutine that yields twice"""
def simple_coro2(a):
    print('-> Started: a = ', a)
    b = yield a
    print('-> Received: b = ', b)
    c = yield a + b
    print('-> Received: c = ', c)

In [8]:
my_coro2 = simple_coro2(14)

In [9]:
from inspect import getgeneratorstate

In [10]:
getgeneratorstate(my_coro2) # The coroutine has not started.

'GEN_CREATED'

In [11]:
"""
Advance coroutine to first yield and print, then yield the value of a and wait
for a value to be assigned to b.
"""
next(my_coro2)

-> Started: a =  14


14

In [12]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [13]:
"""
Send 28 to the suspended coroutine. The yield evaluates to 28 and that value is
bound to b. The print statement is evaluated and the value of `a + b` = 28 + 14 = 42
is yielded and execution is suspended.
"""
my_coro2.send(28)

-> Received: b =  28


42

In [14]:
"""
Send 99 into the coroutine. The yield expression evaluates to 99, which is bound to c.
The print statement runs and the coroutine terminates, causing the generator to
raise `StopIteration`.
"""
my_coro2.send(99)

-> Received: c =  99


StopIteration: 

In [15]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

In [16]:
"""
Computing a running average using a coroutine
"""
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [17]:
coro_avg = averager()

In [18]:
next(coro_avg)

In [19]:
coro_avg.send(10)

10.0

In [20]:
coro_avg.send(30)

20.0

In [21]:
coro_avg.send(5)

15.0

## Decorators for coroutine priming
- Priming a corouting is calling `next(coroutine)` before calling `coroutine.send(...)`.

In [22]:
"""
Decorator for priming a coroutine
"""
from functools import wraps

def coroutine(func):
    """Decorator primes `func` before advancing it to first yield"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [23]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [24]:
coro_avg = averager()

In [25]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [26]:
coro_avg.send(10)

10.0

In [27]:
coro_avg.send(20)

15.0

In [28]:
coro_avg.send(10)

13.333333333333334

----

## Using `yield from`

- `yield from` is similar to `await` in other languages.
- When `gen` calls `yield from subgen()`, the `subgen` takes over and will yield values to the caller of `gen`, i.e, the caller of `gen` will directly drive `subgen` while `gen` will be blocked waiting until `subgen` terminates.

> `yield from` can be used as a shortcut to yield from a for loop

In [1]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

In [2]:
list(gen())

['A', 'B', 1, 2]

In [3]:
def gen_yield_from():
    yield from 'AB'
    yield from range(1, 3)

In [4]:
list(gen_yield_from())

['A', 'B', 1, 2]

- Main feature of `yield from` is to open bidirectional channel from the outermost caller to the innermost subgenerator so that values can be yielded back and forth directly from them and exceptions thrown all the way in without adding a lot of exception handling boilerplate code in the intermediate coroutines.

#### Terms as  used in **_PEP 380: Syntax for Delegating to a Subgenerator_**
- **`delegating generator`** - The generator function containing the `yield from <iterable>` expression.
- **`subgenerator`** - The generator obtained from the `<iterable>` part of the `yield from` epxression.
- **`caller`** - Refers to the client code that calls the delegating generator.

In [5]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


# The subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)


# The delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager()


# The client code, a.k.a the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)

        # The next line is important since if you don't terminate the subgenerator,
        # the delegating generator is suspended forever at the yield from statement.
        # This doesn't prevent the program from making progress since yield from, like
        # yield, transfers control back to the client code. It does, however, mean that
        # a task will not complete.
        group.send(None)
    print(results)
    report(results)


# output results
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


if __name__ == '__main__':
    main(data)


{'boys;m': Result(count=9, average=1.3888888888888888), 'girls;m': Result(count=10, average=1.4279999999999997), 'girls;kg': Result(count=10, average=42.040000000000006), 'boys;kg': Result(count=9, average=40.422222222222224)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
